# 02 - Data from the Web

## Deadline
Wednesday October 25, 2017 at 11:59PM

## Important Notes
* Make sure you push on GitHub your Notebook with all the cells already evaluated (i.e., you don't want your colleagues to generate unnecessary Web traffic during the peer review)
* Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
* Please write all your comments in English, and use meaningful variable names in your code.

## Background
In this homework we will extract interesting information from www.topuniversities.com and www.timeshighereducation.com, two platforms that maintain a global ranking of worldwide universities. This ranking is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need!
You are not allowed to download manually the entire ranking -- rather you have to understand how the server loads it in your browser. For this task, Postman with the Interceptor extension can help you greatly. We recommend that you watch this [brief tutorial](https://www.youtube.com/watch?v=jBjXVrS8nXs&list=PLM-7VG-sgbtD8qBnGeQM5nvlpqB_ktaLZ&autoplay=1) to understand quickly how to use it.

## Assignment
1. Obtain the 200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the [details page](https://www.topuniversities.com/universities/ecole-polytechnique-fÃ©dÃ©rale-de-lausanne-epfl).
Store the resulting dataset in a pandas DataFrame and answer the following questions:
    - Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
    - Answer the previous question aggregating the data by (c) country and (d) region.
    
 Plot your data using bar charts and describe briefly what you observed.

2. Obtain the 200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)). Repeat the analysis of the previous point and discuss briefly what you observed.

3. Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

4. Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

5. Can you find the best university taking in consideration both rankings? Explain your approach.

Hints:
- Keep your Notebook clean and don't print the verbose output of the requests if this does not add useful information for the reader.
- In case of tie, use the order defined in the webpage.

##  Textual description of your thought process, the assumptions you made, and the solution you plan to implement!

### Thought process:

www.topuniversities.com 
1. get https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508228964836
    a. this is a json file
2. obtain name rank country region
3. get https://www.topuniversities.com/universities/ecole-polytechnique-f%C3%A9d%C3%A9rale-de-lausanne-epfl
    a. this is a html file
4. get number of faculty -- international and total, number of students -- international and total

### analysis
1. Faculty : student ratio
2. international student : total student ratio



### Our assumptions:
Location in times_university is country.
There is no international faculty in Indian Institute of Science (IISc) Bangalore (top_uni).


### Implementation solution:

In [117]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

### Question 1.

We first crawl data from the webpages, including the details page of universities we are interested in, and then put it into a dataframe.

In [71]:
# Request the main page containing the rank
req1 = requests.get('https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508228964836')
body = req1.json()
body_data = body['data']

# The fields we are interested in
# Global data
dict_fields = {'Name':    'title',
               'Rank':    'rank_display',
               'Country': 'country',
               'Region':  'region'}
# Detail pages
dict_class_names = {'No. of International Faculty':  'inter faculty',
                    'No. of Total Faculty':          'total faculty',
                    'No. of International Students': 'total inter',
                    'No. of Total Students':         'total student'}

# The helper function that gets information from details pages
def get_numbers(soup, class_name):
    try:
        num = float(soup.find(class_=class_name).find(class_='number').string[1:-1].replace(",", ""))
    except:
        # If there's no data, print an error and fall back to NaN
        num = float("NaN")
        school_name = soup.find(class_ = "qs-profile-2 content panel-panel").find_next('h1').string
        print("Error in " + school_name + ": Can't find " + class_name)
    return num

# Add data of each university one by one
rank_list = []
for i in range(200):
    # Request the details page
    details = requests.get('https://www.topuniversities.com' + body_data[i]['url'])
    html_body = details.text
    # Parse the HTML
    soup = BeautifulSoup(html_body, 'html.parser')
    
    dict_school = {}
    for field, field_name in dict_fields.items():
        dict_school[field] = body_data[i][field_name]
    for field, class_name in dict_class_names.items():
        dict_school[field] = get_numbers(soup, class_name)
    
    rank_list.append(dict_school)

top_ranking_universities = pd.DataFrame.from_dict(rank_list)

Error in New York University (NYU): Can't find inter faculty
Error in New York University (NYU): Can't find total faculty
Error in New York University (NYU): Can't find total inter
Error in New York University (NYU): Can't find total student


In [72]:
top_ranking_universities.head()

,Country,Name,No. of International Faculty,No. of International Students,No. of Total Faculty,No. of Total Students,Rank,Region
0,United States,Massachusetts Institute of Technology (MIT),1679.0,3717.0,2982.0,11067.0,1,North America
1,United States,Stanford University,2042.0,3611.0,4285.0,15878.0,2,North America
2,United States,Harvard University,1311.0,5266.0,4350.0,22429.0,3,North America
3,United States,California Institute of Technology (Caltech),350.0,647.0,953.0,2255.0,4,North America
4,United Kingdom,University of Cambridge,2278.0,6699.0,5490.0,18770.0,5,Europe


The New York University's page has no data on the number of faculty and students. The Indian Institute of Science has no international faculty.

From the dataframe obtained, we compute the ratio of factulty to student and international student to total student.

In [73]:
# Add new columns
top_ranking_universities['Faculty : Students'] = \
    top_ranking_universities['No. of Total Faculty']/ \
    top_ranking_universities['No. of Total Students']
top_ranking_universities['International Students : Total Students'] = \
    top_ranking_universities['No. of International Students']/ \
    top_ranking_universities['No. of Total Students']

top_ranking_universities.head()

,Country,Name,No. of International Faculty,No. of International Students,No. of Total Faculty,No. of Total Students,Rank,Region,Faculty : Students,International Students : Total Students
0,United States,Massachusetts Institute of Technology (MIT),1679.0,3717.0,2982.0,11067.0,1,North America,0.269450,0.335863
1,United States,Stanford University,2042.0,3611.0,4285.0,15878.0,2,North America,0.269870,0.227422
2,United States,Harvard University,1311.0,5266.0,4350.0,22429.0,3,North America,0.193945,0.234785
3,United States,California Institute of Technology (Caltech),350.0,647.0,953.0,2255.0,4,North America,0.422616,0.286918
4,United Kingdom,University of Cambridge,2278.0,6699.0,5490.0,18770.0,5,Europe,0.292488,0.356899


### Question 1 (a): Which are the best universities in term of ratio between faculty members and students?

In [76]:
# Sort the dataframe by faculty-student ratio in descending order 
top_uni_by_facstud = top_ranking_universities.sort_values(
    by = 'Faculty : Students',
    ascending = False
)
top_uni_by_facstud[['Rank', 'Name', 'Faculty : Students']].head(10)

,Rank,Name,Faculty : Students
3,4,California Institute of Technology (Caltech),0.422616
15,16,Yale University,0.398323
5,6,University of Oxford,0.342292
4,5,University of Cambridge,0.292488
16,17,Johns Hopkins University,0.276353
1,2,Stanford University,0.269870
0,1,Massachusetts Institute of Technology (MIT),0.269450
185,186,University of Rochester,0.266604
18,19,University of Pennsylvania,0.266437
17,18,Columbia University,0.247115


In [ ]:
# Plot the faculty:student ratio
fig = plt.figure()
ax = fig.add_axes((0,0,1,0.96))

fig.suptitle('Faculty to Student Ratio', fontsize=40)
fig.set_size_inches(20, 40)

ax.barh(np.arange(200), top_uni_by_facstud['Faculty : Student'], align='center')
ax.set_xlabel('Ratio', fontsize=20)
ax.set_yticks(np.arange(200))
ax.set_yticklabels(top_uni_by_facstud.Name)
ax.invert_yaxis()  # labels read top-to-bottom
ax.margins(x=1e-2,y=1e-2)

plt.show()

### Question 1 (b): Which are the best universities in term of ratio of international students?

In [78]:
# Sort by the desired column
top_uni_by_interstud = top_ranking_universities.sort_values(
    by = 'International Students : Total Students',
    ascending = False
)
top_uni_by_interstud[['Rank', 'Name', 'International Students : Total Students']].head(10)

,Rank,Name,International Students : Total Students
34,35,London School of Economics and Political Scien...,0.691393
11,12,Ecole Polytechnique Fédérale de Lausanne (EPFL),0.570047
7,8,Imperial College London,0.543567
198,200,Maastricht University,0.502533
47,=47,Carnegie Mellon University,0.478062
6,7,UCL (University College London),0.477928
91,92,University of St Andrews,0.457955
41,=41,The University of Melbourne,0.427434
126,127,Queen Mary University of London,0.421816
25,26,The University of Hong Kong,0.407144


In [ ]:
# Plot the international:total student ratio
fig = plt.figure()
ax = fig.add_axes((0,0,1,0.96))

fig.suptitle('International to Total Student Ratio', fontsize=40)
fig.set_size_inches(20, 40)

ax.barh(np.arange(200), top_uni_by_interstud['International Student : Total Student'], align='center')
ax.set_xlabel('Ratio', fontsize=20)
ax.set_yticks(np.arange(200))
ax.set_yticklabels(top_uni_by_facstud.Name)
ax.invert_yaxis()  # labels read top-to-bottom
ax.margins(x=1e-2,y=1e-2)

plt.show()

### Question 1 (c): Answer the previous question aggregating the data by country.

We computed the average of the data of all universities in each country.

In [163]:
# Group data by country and compute the average of all fields
by_country = top_ranking_universities.groupby('Country').mean()

# Sort the entries of each group
country_facstud = by_country.sort_values(
    by = 'Faculty : Students',
    ascending = False
)

country_interstud = by_country.sort_values(
    by = 'International Students : Total Students',
    ascending = False
)

country_facstud[['Faculty : Students']].head()

,Faculty : Students
Country,
Russia,0.221910
Denmark,0.186580
Saudi Arabia,0.175828
Singapore,0.162279
Japan,0.155840


In [164]:
country_interstud[['International Students : Total Students']].head()

,International Students : Total Students
Country,
United Kingdom,0.351308
Australia,0.346878
Switzerland,0.313816
Hong Kong,0.312148
Austria,0.306095


### Question 1 (d): Answer the previous question aggregating the data by region.

Same as above, but grouping by region.

In [165]:
# Group data by country and compute the average of all fields
by_region = top_ranking_universities.groupby('Region').mean()

# Sort the entries of each group
region_facstud = by_region.sort_values(
    by = 'Faculty : Students',
    ascending = False
)

region_interstud = by_region.sort_values(
    by = 'International Students : Total Students',
    ascending = False
)

region_facstud[['Faculty : Students']]

,Faculty : Students
Region,
North America,0.144831
Asia,0.134673
Europe,0.120003
Latin America,0.096779
Africa,0.088450
Oceania,0.075003


In [166]:
region_interstud[['International Students : Total Students']]

,International Students : Total Students
Region,
Oceania,0.329077
Europe,0.245932
North America,0.202437
Africa,0.169703
Asia,0.132394
Latin America,0.071751


### Question 2: Obtain the 200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)). Repeat the analysis of the previous point and discuss briefly what you observed.

The data is located at https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json

The fields we're interested in are
* rank
* name
* aliases
* location
* stats_number_students
* stats_student_staff_ratio
* stats_pc_intl_students

In [ ]:
# Fetch the data
req_times = requests.get('https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json')
body_times = req_times.json()
body_times_data = body_times['data']

The Times Higher Education ranking defines "Staff" as "those involved in teaching or research" which matches our definition of "Faculty"

In [139]:
# Parse the data and build a dataframe
rank_list_times = []

for i in range(200):
    name = body_times_data[i]['name']
    aliases = body_times_data[i]['aliases']
    rank = body_times_data[i]['rank'].strip('=')
    location = body_times_data[i]['location']
    
    # Python doesn't like commas as thousands separators, remove them
    num_students = int(body_times_data[i]['stats_number_students'].replace(",",""))
    faculty_students = 1 / float(body_times_data[i]['stats_student_staff_ratio'])
    
    # Same for percentage signs. In addition we need to remove divide by 100
    perc_international = int(body_times_data[i]['stats_pc_intl_students'].strip("%"))/100.0
    
    rank_list_times.append({'Name': name,
                            'Aliases': aliases,
                            'Rank': rank,
                            'Country': location,
                            'No. of Total Students': num_students,
                            'Faculty : Students': faculty_students,
                            'International Students : Total Students': perc_international})
    
times_universities_no_region = pd.DataFrame.from_dict(rank_list_times)

This dataset lacks a 'Region' column, but we can add one using the data from the topuniversities ranking.

In [144]:
tu_region_country = top_ranking_universities[['Region', 'Country']].drop_duplicates()
# Left join because we want all the rows that were originally in the times dataframe and only those
times_with_region = pd.merge(times_universities_no_region, tu_region_country, how='left')

# Check for missing data
times_with_region[times_with_region.Region.isnull()]

,Aliases,Country,Faculty : Students,International Students : Total Students,Name,No. of Total Students,Rank,Region
178,University of Luxembourg,Luxembourg,0.068493,0.57,University of Luxembourg,4969,179,NaN
193,Lomonosov Moscow State University,Russian Federation,0.136986,0.22,Lomonosov Moscow State University,29236,194,NaN


We need to add the missing regions manually.

In [145]:
trc = tu_region_country.append([
    {'Country': 'Luxembourg', 'Region': 'Europe'},
    {'Country': 'Russian Federation', 'Region': 'Europe'}
])
times_universities = pd.merge(times_universities_no_region, trc, how='left')

# Check if there are still any missing values
times_universities.Region.isnull().any()

False

In [143]:
times_universities.head()

,Aliases,Country,Faculty : Students,International Students : Total Students,Name,No. of Total Students,Rank,Region
0,University of Oxford,United Kingdom,0.089286,0.38,University of Oxford,20409,1,Europe
1,University of Cambridge,United Kingdom,0.091743,0.35,University of Cambridge,18389,2,Europe
2,California Institute of Technology caltech,United States,0.153846,0.27,California Institute of Technology,2209,3,North America
3,Stanford University,United States,0.133333,0.22,Stanford University,15845,3,North America
4,Massachusetts Institute of Technology,United States,0.114943,0.34,Massachusetts Institute of Technology,11177,5,North America


Even though the two rankings use similar definitions for "Faculty" and "Students", the results appear to be significantly different: for example, the faculty to student ratio is more than four times higher in the topuniversities ranking than in the THE ranking for the university of Oxford.

[QS's definition](http://www.iu.qs.com/university-rankings/indicator-faculty-student/)   
[THE's definition](https://www.timeshighereducation.com/world-university-rankings-2018-table-information)

### Question 2 (a): Which are the best universities in term of ratio between faculty members and students?

For the above reason, this ranking differs significantly from the one obtained from the topuniversities data.

In [147]:
times_uni_by_facstud = times_universities.sort_values(
    by = 'Faculty : Students',
    ascending=False
)

times_uni_by_facstud[['Rank', 'Name', 'Faculty : Students']].head(10)

,Rank,Name,Faculty : Students
105,105,Vanderbilt University,0.303030
109,109,University of Copenhagen,0.243902
153,153,University of Rochester,0.232558
11,12,Yale University,0.232558
12,13,Johns Hopkins University,0.232558
97,98,Emory University,0.227273
16,17,Duke University,0.222222
114,115,École Polytechnique,0.196078
183,184,Scuola Normale Superiore di Pisa,0.192308
101,100,University of Pittsburgh,0.169492


### Question 2 (b): Which are the best universities in term of ratio of international students?

In [150]:
times_uni_by_facstud = times_universities.sort_values(
    by = 'International Students : Total Students',
    ascending=False
)

times_uni_by_facstud[['Rank', 'Name', 'International Students : Total Students']].head(10)

,Rank,Name,International Students : Total Students
24,25,London School of Economics and Political Science,0.71
178,179,University of Luxembourg,0.57
37,38,École Polytechnique Fédérale de Lausanne,0.55
7,8,Imperial College London,0.55
102,103,Maastricht University,0.50
15,16,University College London,0.49
143,143,University of St Andrews,0.48
23,24,Carnegie Mellon University,0.45
120,121,Queen Mary University of London,0.45
39,40,University of Hong Kong,0.42


### Question 2 (c): Answer the previous question aggregating the data by country.

In [151]:
# Group data by country and compute the average of all fields
times_by_country = times_universities.groupby('Country').mean()

# Sort the entries of each group
times_country_facstud = by_country.sort_values(
    by = 'Faculty : Students',
    ascending = False
)

times_country_interstud = by_country.sort_values(
    by = 'International Students : Total Students',
    ascending = False
)

times_country_facstud[['Faculty : Students']].head()

,Faculty : Students
Country,
Denmark,0.160166
Italy,0.138889
Russian Federation,0.136986
Japan,0.132098
United States,0.106390


In [152]:
times_country_interstud[['International Students : Total Students']].head()

,International Students : Total Students
Country,
Luxembourg,0.570000
United Kingdom,0.365484
Hong Kong,0.328000
Switzerland,0.314286
Australia,0.307500


### Question 2 (d): Answer the previous question aggregating the data by region.

In [167]:
# Group data by country and compute the average of all fields
times_by_region = times_universities.groupby('Region').mean()

# Sort the entries of each group
times_region_facstud = by_region.sort_values(
    by = 'Faculty : Students',
    ascending = False
)

times_region_interstud = by_region.sort_values(
    by = 'International Students : Total Students',
    ascending = False
)

times_region_facstud[['Faculty : Students']]

,Faculty : Students
Region,
North America,0.144831
Asia,0.134673
Europe,0.120003
Latin America,0.096779
Africa,0.088450
Oceania,0.075003


In [168]:
times_region_interstud[['International Students : Total Students']]

,International Students : Total Students
Region,
Oceania,0.329077
Europe,0.245932
North America,0.202437
Africa,0.169703
Asia,0.132394
Latin America,0.071751


### Question 3: Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

In [ ]:
import numpy as np
import unicodedata

ranks = pd.merge(top_ranking_universities, times_universities, how='outer', on='Name', suffixes=('_top','_tim'))
mask = ranks[['Rank_top','Rank_tim']].astype(str) != 'nan'
both = ranks[mask.all(axis=1)][['Rank_top','Rank_tim','Name']]
tim_only = ranks[list(mask['Rank_tim']==True) and list(mask['Rank_top']==False)][['Rank_top','Rank_tim','Name']]
top_only = ranks[list(mask['Rank_top']==True) and list(mask['Rank_tim']==False)][['Rank_top','Rank_tim','Name']]
ranks.shape, both.shape, tim_only.shape, top_only.shape

In [ ]:
# There are universities having different "Name" in the two websites.
# Following should be the whole list of unmathed name:
# California Institute of Technology v.s. California Institute of Technology (Caltech)
# ETH Zurich - Swiss... v.s. ETH Zurich – Swiss...
# Ecole Polytechnique v.s. École Polytechnique
# Ecole Polytechnique Fédérale de Lausanne (EPFL) v.s. École Polytechnique Fédérale de Lausanne
# Humboldt University of Berlin v.s. Humboldt-Universität zu Berlin
# King's College London v.s. King’s College London
# London School of Economics and Political Science v.s. London School of Economics and Political Science (LSE)
# Massachusetts Institute of Technology v.s. Massachusetts Institute of Technology (MIT)
# Nanyang Technological University, Singapore v.s. Nanyang Technological University, Singapore (NTU)
# National Taiwan University v.s. National Taiwan University (NTU)
# National University of Singapore v.s. National University of Singapore (NUS)
# New York University v.s. New York University (NYU)
# Pohang University of Science And Technology (POSTECH) v.s. Pohang University of Science and Technology
# Scuola Superiore Sant'Anna Pisa di Studi Universitari e di Perfezionamento v.s. Scuola Superiore Sant’Anna
# Technical University of Berlin v.s. Technische Universität Berlin (TU Berlin)
# Trinity College Dublin v.s. Trinity College Dublin, The University of Dublin
# University of California, Berkeley v.s. University of California, Berkeley (UCB)
# University of California, Los Angeles v.s. University of California, Los Angeles (UCLA)
# University of California, San Diego v.s. University of California, San Diego (UCSD)
# University of California, Santa Barbara v.s. University of California, Santa Barbara (UCSB)
# University of North Carolina at Chapel Hill v.s. University of North Carolina, Chapel Hill
# Wageningen University v.s. Wageningen University & Research
# Washington University in St Louis v.s. Washington University in St. Louis

pd.set_option('display.max_rows', 400)
cols = ranks.columns
(list(zip(np.arange(len(cols)),cols)))
merged = ranks.sort_values(by='Name')[ranks.columns[[6,13,1,10,7,0,11,8,9,15,14,2,3,4,5,12]]]
merged.tail().Name
#merged.to_csv('merged.csv')


In [ ]:
# Dark magic: transform unicode to ascii, get abbriviation. The result is useless
def dark_magic(name):
    name = unicodedata.normalize('NFD', name).encode('ascii', 'ignore')
    name = ''.join(c for c in str(name) if c.isupper())
    return name

print(tim_only.loc[214].Name)
print(dark_magic(tim_only.loc[214].Name))

In [ ]:
top_ranking_universities['Name2'] = top_ranking_universities['Name'].apply(dark_magic)
times_universities['Name2'] = times_universities['Name'].apply(dark_magic)
ranks2 = pd.merge(top_ranking_universities, times_universities, how='outer', on='Name2', suffixes=('_top','_tim'))
mask2 = ranks2[['Rank_top','Rank_tim']].astype(str) != 'nan'
both2 = ranks2[mask2.all(axis=1)][['Rank_top','Rank_tim','Name2']]
tim_only2 = ranks2[list(mask2['Rank_tim']==True) and list(mask2['Rank_top']==False)][['Rank_top','Rank_tim','Name2']]
top_only2 = ranks2[list(mask2['Rank_top']==True) and list(mask2['Rank_tim']==False)][['Rank_top','Rank_tim','Name2']]
ranks2.shape, both2.shape, tim_only2.shape, top_only2.shape

### Question 4: Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

### Question 5: Can you find the best university taking in consideration both rankings? Explain your approach.